In [1]:
import string
import re
import pandas as pd
import os

from utils import basic
from db.get_db_data import GetTableData
from db.config import Config
_Config = Config()

# setup db connection
conn_cur_list = basic.setup_db_connection()
query_path = str(_Config.queries["get_sta_tables"])
query_string = basic.read_query_file(query_path)

# create 
_GetTableData = GetTableData(conn_cur_list[0], conn_cur_list[1])
sta_tables_df = _GetTableData.create_pandas_table(query_string)

print(sta_tables_df.head())

                       table_name
0   sta_euro_dollar_exchange_rate
1                sta_copper_price
2              sta_usa_bond_2year
3  sta_1_year_treasury_rate_yield
4        sta_capacity_utilization


In [2]:
#test_table = sta_tables_df.iloc[1]

In [3]:
query_path = str(_Config.queries["select_table_sample"])
query_string = basic.read_query_file(query_path)
query_string

'select * from __tablename__ order by RANDOM() limit __limit__;'

In [4]:
def get_sta_dateformats(sta_table):
    query = query_string.replace("__tablename__", sta_table)
    query = query.replace("__limit__", "1000")
    df = _GetTableData.create_pandas_table(query)

    # determine the date formats for all table sample cells
    df['datetype'] = df.iloc[:,0].apply(lambda x: basic.string_to_sql_type(str(x)))
    
    # return the most presented date format
    major_dateformat = df['datetype'].value_counts(ascending=False).to_frame().index.values[0]

    return major_dateformat

In [5]:
sta_tables_df["datetype"] = sta_tables_df['table_name'].apply(lambda x: get_sta_dateformats(x))

In [6]:
sta_tables_df

,table_name,datetype
0,sta_euro_dollar_exchange_rate,date__M/D/YYYY
1,sta_copper_price,date__M/D/YYYY
2,sta_usa_bond_2year,"date__Mon DD, YYYY"
3,sta_1_year_treasury_rate_yield,date__M/D/YYYY
4,sta_capacity_utilization,date__M/D/YYYY
5,sta_futures_vix,"date__Mon DD, YYYY"
6,sta_heating_oil_prices,date__M/D/YYYY
7,sta_industrial_production,date__M/D/YYYY
8,sta_palladium_prices,date__M/D/YYYY
9,sta_sp500,date__YYYY-MM-DD


In [7]:
# retrieve one M/D/YYYY table as pd dataframe

from db.get_dbtable_data import get_dbtable_data

sta_tablename = 'sta_putcall_vix'

test_dm_table = get_dbtable_data(sta_tablename)
test_dm_table

,date,vixputcallratio,vixputvolume,vixcallvolume,totalvixoptionsvolume
0,2/24/2006,1.18,5095,4328,9423
1,2/27/2006,0.01,71,6996,7067
2,2/28/2006,1.75,5400,3086,8486
3,3/1/2006,0.7,1252,1779,3031
4,3/2/2006,0.18,2690,15156,17846
...,...,...,...,...,...
3422,9/30/2019,0.81,109200,134540,243740
3423,10/1/2019,0.97,184221,190859,375080
3424,10/2/2019,0.56,271592,488159,759751
3425,10/3/2019,0.46,126939,273016,399955


In [8]:
# new data frame with split value columns 
new = test_dm_table.iloc[:,0].str.split("/", n = 3, expand = True) 
new



,0,1,2
0,2,24,2006
1,2,27,2006
2,2,28,2006
3,3,1,2006
4,3,2,2006
...,...,...,...
3422,9,30,2019
3423,10,1,2019
3424,10,2,2019
3425,10,3,2019


In [9]:
# making separate month column from new data frame 
test_dm_table["month"] = new[0] 
  
# making separate day column from new data frame 
test_dm_table["day"] = new[1] 

# making year column from new data frame 
test_dm_table["year"] = new[2] 
test_dm_table

,date,vixputcallratio,vixputvolume,vixcallvolume,totalvixoptionsvolume,month,day,year
0,2/24/2006,1.18,5095,4328,9423,2,24,2006
1,2/27/2006,0.01,71,6996,7067,2,27,2006
2,2/28/2006,1.75,5400,3086,8486,2,28,2006
3,3/1/2006,0.7,1252,1779,3031,3,1,2006
4,3/2/2006,0.18,2690,15156,17846,3,2,2006
...,...,...,...,...,...,...,...,...
3422,9/30/2019,0.81,109200,134540,243740,9,30,2019
3423,10/1/2019,0.97,184221,190859,375080,10,1,2019
3424,10/2/2019,0.56,271592,488159,759751,10,2,2019
3425,10/3/2019,0.46,126939,273016,399955,10,3,2019


In [10]:
def fill_datecol_with_zeros(day_o_month):
    if len(day_o_month) == 1:
        day_o_month = '0' + day_o_month
    return day_o_month

In [11]:
test_dm_table['month_o'] = test_dm_table['month'].apply(lambda x: fill_datecol_with_zeros(x))
test_dm_table['day_o'] = test_dm_table['day'].apply(lambda x: fill_datecol_with_zeros(x))

test_dm_table

,date,vixputcallratio,vixputvolume,vixcallvolume,totalvixoptionsvolume,month,day,year,month_o,day_o
0,2/24/2006,1.18,5095,4328,9423,2,24,2006,02,24
1,2/27/2006,0.01,71,6996,7067,2,27,2006,02,27
2,2/28/2006,1.75,5400,3086,8486,2,28,2006,02,28
3,3/1/2006,0.7,1252,1779,3031,3,1,2006,03,01
4,3/2/2006,0.18,2690,15156,17846,3,2,2006,03,02
...,...,...,...,...,...,...,...,...,...,...
3422,9/30/2019,0.81,109200,134540,243740,9,30,2019,09,30
3423,10/1/2019,0.97,184221,190859,375080,10,1,2019,10,01
3424,10/2/2019,0.56,271592,488159,759751,10,2,2019,10,02
3425,10/3/2019,0.46,126939,273016,399955,10,3,2019,10,03


In [12]:
test_dm_table['date'] = test_dm_table['month_o'] + '/' + test_dm_table['day_o'] + '/' + test_dm_table['year']
test_dm_table.drop(['month', 'month_o', 'day_o', 'year', 'day'], axis=1, inplace=True)


In [13]:
test_dm_table

,date,vixputcallratio,vixputvolume,vixcallvolume,totalvixoptionsvolume
0,02/24/2006,1.18,5095,4328,9423
1,02/27/2006,0.01,71,6996,7067
2,02/28/2006,1.75,5400,3086,8486
3,03/01/2006,0.7,1252,1779,3031
4,03/02/2006,0.18,2690,15156,17846
...,...,...,...,...,...
3422,09/30/2019,0.81,109200,134540,243740
3423,10/01/2019,0.97,184221,190859,375080
3424,10/02/2019,0.56,271592,488159,759751
3425,10/03/2019,0.46,126939,273016,399955


In [14]:
# mach die alte tabelle leer
from db.write_table import write_table

write_table(test_dm_table, sta_tablename)

# füge die neuen bereinigten Daten ein

INFO:root: dropping table sta_putcall_vix if it exists
